#  Instalar dependências

In [1]:
!pip install -q ultralytics kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.1 MB/s eta 0:00:00


# Fazer upload do kaggle.json

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"luisthedevmagician","key":"c47963d0936c6df20a8ee23348fa030f"}'}

#  Configurar Kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Baixar dataset WIDER Face formatado para YOLO

In [4]:
!kaggle datasets download -d lylmsc/wider-face-for-yolo-training
!unzip wider-face-for-yolo-training.zip -d wider_face

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: wider_face/labels/wider_5499.txt  
  inflating: wider_face/labels/wider_55.txt  
  inflating: wider_face/labels/wider_550.txt  
  inflating: wider_face/labels/wider_5500.txt  
  inflating: wider_face/labels/wider_5501.txt  
  inflating: wider_face/labels/wider_5502.txt  
  inflating: wider_face/labels/wider_5503.txt  
  inflating: wider_face/labels/wider_5504.txt  
  inflating: wider_face/labels/wider_5505.txt  
  inflating: wider_face/labels/wider_5506.txt  
  inflating: wider_face/labels/wider_5507.txt  
  inflating: wider_face/labels/wider_5508.txt  
  inflating: wider_face/labels/wider_5509.txt  
  inflating: wider_face/labels/wider_551.txt  
  inflating: wider_face/labels/wider_5510.txt  
  inflating: wider_face/labels/wider_5511.txt  
  inflating: wider_face/labels/wider_5512.txt  
  inflating: wider_face/labels/wider_5513.txt  
  inflating: wider_face/labels/wider_5514.txt  
  inflating: wider_face/labels/wi

#  Reorganizar diretório (já vem no padrão YOLO: images/ e labels/)

In [5]:
import os
from pathlib import Path
import shutil

# Mover para estrutura correta se necessário
base = Path("wider_face")
if not (base/"images/train").exists():
    # Criar estrutura padrão YOLO
    os.makedirs(base/"images/train", exist_ok=True)
    os.makedirs(base/"images/val", exist_ok=True)
    os.makedirs(base/"labels/train", exist_ok=True)
    os.makedirs(base/"labels/val", exist_ok=True)

    # Distribuir 80% treino, 20% validação
    from sklearn.model_selection import train_test_split
    all_images = list((base/"images").glob("*.jpg"))
    train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

    for img in train_imgs:
        shutil.move(str(img), base/"images/train"/img.name)
        label = base/"labels"/img.with_suffix('.txt').name
        if Path(label).exists():
            shutil.move(str(label), base/"labels/train"/Path(label).name)

    for img in val_imgs:
        shutil.move(str(img), base/"images/val"/img.name)
        label = base/"labels"/img.with_suffix('.txt').name
        if Path(label).exists():
            shutil.move(str(label), base/"labels/val"/Path(label).name)


#  Criar arquivo data.yaml

In [6]:
yaml_text = """
path: /content/wider_face
train: images/train
val: images/val

names:
  0: face
"""
with open("data.yaml", "w") as f:
    f.write(yaml_text)

# Treinar o modelo YOLOv8

In [7]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # Pode trocar por yolov8s.pt para mais precisão

model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="yolov8-face",
    pretrained=True
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 397MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8-face, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

100%|██████████| 755k/755k [00:00<00:00, 83.1MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      50/50      17.4G      1.342     0.6291     0.9651        134        640: 100%|██████████| 644/644 [00:57<00:00, 11.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:10<00:00,  7.84it/s]


                   all       2575      35407      0.823      0.514      0.595      0.321

50 epochs completed in 0.959 hours.
Optimizer stripped from runs/detect/yolov8-face/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8-face/weights/best.pt, 6.2MB

Validating runs/detect/yolov8-face/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:11<00:00,  6.92it/s]


                   all       2575      35407      0.821      0.515      0.595      0.321
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/yolov8-face


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb38ad8fdd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# Avaliar desempenho no conjunto de validação

In [8]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3101.0±656.4 MB/s, size: 210.8 KB)


val: Scanning /content/wider_face/labels/val.cache... 2576 images, 1 backgrounds, 1 corrupt: 100%|██████████| 2576/2576 [00:00<?, ?it/s]

val: /content/wider_face/images/val/wider_10969.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0254]
val: /content/wider_face/images/val/wider_12381.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 161/161 [00:12<00:00, 12.46it/s]


                   all       2575      35407      0.825      0.515      0.597      0.323
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/yolov8-face2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb388b0c290>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,   

# Visualizar predições em imagens de validação

In [9]:
results = model.predict(source="wider_face/images/val", conf=0.3, save=True)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2576 /content/wider_face/images/val/wider_10.jpg: 480x640 2 faces, 45.8ms
image 2/2576 /content/wider_face/images/val/wider_10000.jpg: 640x448 1 face, 44.8ms
image 3/2576 /content/wider_face/images/val/wider_10009.jpg: 640x640 1 face, 7.5ms
image 4/2576 /content/wider_face/images/val/wider_10022.jpg: 512x640 6 faces, 44.5ms
image 5/2576 /content/wider_face/images/val/wider_10025.jpg: 640x448 1 face, 7.7ms
image 6/2576 /content/wider_face/images